# Setting up<a href="#Setting-up" class="anchor-link"></a>

In [ ]:
 !pip install rpy2

In [ ]:
    #Set up libraries

    # Use the os package to interact with the environment
    import os

    # Bring in Pandas for Dataframe functionality
    import pandas as pd

    # numpy for basics
    import numpy as np

    import sys
    import subprocess
    import glob
    from functools import partial 
    from os import chdir
    import io
    import time
    import matplotlib.pyplot as plt
    import seaborn as sns


    # Use StringIO for working with file contents
    from io import StringIO

    # Enable IPython to display matplotlib graphs
    import matplotlib.pyplot as plt
    %matplotlib inline

    # Enable interaction with the FireCloud API
    from firecloud import api as fapi

    # Import the iPython HTML rendering for displaying links to Google Cloud Console
    from IPython.core.display import display, HTML

    # Import urllib modules for building URLs to Google Cloud Console
    import urllib.parse

    # BigQuery for querying data
    from google.cloud import bigquery

    # Installing and importing rpy2 
    import rpy2.rinterface

In [ ]:
%load_ext rpy2.ipython

In [ ]:
#Setting paths

BILLING_PROJECT_ID = os.environ['GOOGLE_PROJECT']
WORKSPACE_NAMESPACE = os.environ['WORKSPACE_NAMESPACE']
WORKSPACE_NAME = os.environ['WORKSPACE_NAME']
WORKSPACE_BUCKET = os.environ['WORKSPACE_BUCKET']

WORKSPACE_ATTRIBUTES = fapi.get_workspace(WORKSPACE_NAMESPACE, WORKSPACE_NAME).json().get('workspace',{}).get('attributes',{})

GS_RELEASE_PATH = 'gs://amp-pd-data/releases/2021_v2-5release_0510'
GS_CLINICAL_RELEASE_PATH = f'{GS_RELEASE_PATH}/clinical/'
GS_MUTATION_RELEASE_PATH = 'gs://amp-pd-data-tier2/releases/2021_v2-5release_0510'

GS_WGS_RELEASE_PATH = 'gs://amp-pd-genomics/releases/2021_v2-5release_0510/wgs'
GS_WGS_RELEASE_PLINK_PATH = os.path.join(GS_WGS_RELEASE_PATH, 'plink')
GS_WGS_RELEASE_GATK_PATH = os.path.join(GS_WGS_RELEASE_PATH, 'gatk')


BQ_RELEASE_DATASET = 'amp-pd-research.2021_v2-5release_0510'


print(BILLING_PROJECT_ID)
print(GS_CLINICAL_RELEASE_PATH)
print(GS_WGS_RELEASE_PLINK_PATH)
print(GS_WGS_RELEASE_GATK_PATH)

In [ ]:
# # Utility routine for printing a shell command before executing it
def shell_do(command):
    print(f'Executing: {command}')
    !$command

def shell_return(command):
    print(f'Executing: {command}', file=sys.stderr)
    output = !$command
    return '\n'.join(output)


# Utility routine for display a message and a link
def display_html_link(description, link_text, url):
    html = f'''
    <p>
    </p>
    <p>
    {description}
    <a target=_blank href="{url}">{link_text}</a>.
    </p>
    '''

    display(HTML(html))
    
# Utility routines for reading files from Google Cloud Storage
def gcs_read_file(path):
    """Return the contents of a file in GCS"""
    contents = !gsutil -u {BILLING_PROJECT_ID} cat {path}
    return '\n'.join(contents)
    
def gcs_read_csv(path, sep=None):
    """Return a DataFrame from the contents of a delimited file in GCS"""
    return pd.read_csv(StringIO(gcs_read_file(path)), sep=sep, engine='python')

# Utility routine for displaying a message and link to Cloud Console
def link_to_cloud_console_gcs(description, link_text, gcs_path):
    url = '{}?{}'.format(
        os.path.join('https://console.cloud.google.com/storage/browser',
                     gcs_path.replace("gs://","")),
        urllib.parse.urlencode({'userProject': BILLING_PROJECT_ID}))

    display_html_link(description, link_text, url)
    
# Get the data from a query
def bq_query(query):
    print(f'Executing: {query}', file=sys.stderr)
    return pd.read_gbq(query, project_id=BILLING_PROJECT_ID, dialect='standard')

# Utility routine for displaying a message and link to Cloud Console
def link_to_cloud_console_gcs(description, link_text, gcs_path):
    url = '{}?{}'.format(
        os.path.join('https://console.cloud.google.com/storage/browser',
                     gcs_path.replace("gs://","")),
        urllib.parse.urlencode({'userProject': BILLING_PROJECT_ID}))

    display_html_link(description, link_text, url)
    
# Utility routine for displaying a message and link to Cloud Console
def link_to_cloud_console_bq(description, link_text, bq_dataset, bq_table=None):
    project, dataset = bq_dataset.split('.', 1)
    if bq_table:
        page_params = {'page': 'table', 'p': project, 'd': dataset, 't': bq_table}
    else:
        page_params = {'page': 'dataset', 'p': project, 'd': dataset}
    
    url = '{}?{}'.format(
        'https://console.cloud.google.com/bigquery',
        urllib.parse.urlencode(page_params))

    display_html_link(description, link_text, url)    

# Utility routine for printing a query before executing it
def bq_query(query):
    """Return the contents of a query against BigQuery"""
    return pd.read_gbq(
        query,
        project_id=BILLING_PROJECT_ID,
        dialect='standard')

In [ ]:
%%R
if (!require(tidyverse)) install.packages('tidyr')
if (!require(data.table)) install.packages('data.table')
if (!require(dplyr)) install.packages('dplyr')
if (!require(plyr)) install.packages('plyr')
if (!require(ggplot2)) install.packages('ggplot2')
if (!require(caret)) install.packages('caret')


# Load the necessary packages 
library(tidyr)
library(data.table)
library(dplyr)
library(plyr)
library(ggplot2)
library(caret)

# Preparing clinical files<a href="#Preparing-clinical-files" class="anchor-link"></a>

In [ ]:
# Importing everyone with clinical data
demographics_df = gcs_read_csv(os.path.join(GS_CLINICAL_RELEASE_PATH, 'Demographics.csv'))
demographics_df.rename(columns = {'participant_id':'ID'}, inplace = True)
demographics_df.info()

In [ ]:
# Recoding sex
conditions = [
     (demographics_df['sex'] == "Male"),
     (demographics_df['sex'] == "Female")]
choices = [1,2]

demographics_df['SEX'] = np.select(conditions, choices, default=None).astype(np.int64)
demographics_df.info()

In [ ]:
# Clean up and drop the columns we don't need anymore 
demographics_baseline_clean_df = demographics_df.drop(columns=['GUID',
                                                               'ethnicity',
                                                               'sex',
                                                               'race',
                                                               'age_at_baseline',
                                                               'education_level_years',
                                                               'visit_month',
                                                               'visit_name']).copy()

demographics_baseline_clean_df.info()

In [ ]:
# Identify mutation carriers
mutation_df = gcs_read_csv(os.path.join(GS_MUTATION_RELEASE_PATH, 'amp_pd_participant_mutations.csv'))
mutation_df.info()

In [ ]:
# Identify LRRK2 carriers
mutation_lrrk2_df = mutation_df[(mutation_df.has_known_LRRK2_mutation_in_WGS == "Yes")]
mutation_lrrk2_df.info()

In [ ]:
# Renaming columns
mutation_lrrk2_df = mutation_lrrk2_df[['participant_id']].copy()
mutation_lrrk2_df.columns = ['ID']
mutation_lrrk2_df['FID'] = mutation_lrrk2_df['ID']

In [ ]:
pd_case_control_df = gcs_read_csv(os.path.join(GS_RELEASE_PATH, 'amp_pd_case_control.csv'))
pd_case_control_df.info()

In [ ]:
# Subset baseline and latest diagnosis 
pd_case_control_diagnosis_df = pd_case_control_df[['participant_id','diagnosis_latest']].copy()
pd_case_control_diagnosis_df.columns = ['ID', 'LATEST_DIAGNOSIS']
pd_case_control_diagnosis_df.info()
#Total of 10 772 people

pd_case_control_diagnosis_df['LATEST_DIAGNOSIS'].value_counts()

In [ ]:
# Identify PD
PD_df = pd_case_control_diagnosis_df[(pd_case_control_diagnosis_df.LATEST_DIAGNOSIS == "Parkinson's Disease") | (pd_case_control_diagnosis_df.LATEST_DIAGNOSIS == "Idiopathic PD")].copy()
PD_df.info()

In [ ]:
# Identify controls
Control_df = pd_case_control_diagnosis_df[(pd_case_control_diagnosis_df.LATEST_DIAGNOSIS == "No PD Nor Other Neurological Disorder")].copy()
Control_df.info()

In [ ]:
%%R -i mutation_lrrk2_df

LRRK2 <- mutation_lrrk2_df

summary(LRRK2)

In [ ]:
%%R -i PD_df 

# PD with LRRK2 mutation
PD <- PD_df

LRRK2_PD <- inner_join(PD_df, LRRK2, by = "ID") %>%
mutate(DIAGNOSIS = "LRRK2_PD", ID = FID) %>%
dplyr::rename(IID = ID) %>%
select(FID, IID, DIAGNOSIS) %>%
distinct(FID, .keep_all = TRUE)

summary(LRRK2_PD)

In [ ]:
%%R

# PD without LRRK2 mutation
LRRK2_NOPD <- anti_join(LRRK2, PD_df, by = "ID") %>%
mutate(DIAGNOSIS = "LRRK2_NOPD", ID = FID) %>%
dplyr::rename(IID = ID) %>%
select(FID, IID, DIAGNOSIS) %>%
distinct(FID, .keep_all = TRUE)


summary(LRRK2_NOPD)

In [ ]:
%%R -i Control_df

# Controls
Controls <- Control_df

Controls <- Controls %>%
mutate(DIAGNOSIS = "CONTROL")%>%
dplyr::rename(IID = ID) %>%
mutate(FID = IID) %>%
select(FID, IID, DIAGNOSIS) %>%
distinct(FID, .keep_all = TRUE)


summary(Controls)

In [ ]:
%%R 

CLIN <- full_join(LRRK2_PD, LRRK2_NOPD, by = c("IID", "FID", "DIAGNOSIS"))
CLIN2 <- full_join(CLIN, Controls, by = c("IID", "FID", "DIAGNOSIS"))

In [ ]:
%%R -i demographics_baseline_clean_df

SEX <- demographics_baseline_clean_df %>%
dplyr::rename(IID = ID)

In [ ]:
%%R
CLIN3 <- inner_join(CLIN2, SEX, by = "IID") %>%
mutate(DIAGNOSIS1 = case_when(DIAGNOSIS == "CONTROL" ~ 0,
                             DIAGNOSIS == "LRRK2_PD" ~ 1,
                             DIAGNOSIS == "LRRK2_NOPD" ~ 2))

In [ ]:
%%R
ID_KEEP <- CLIN3 %>%
select(FID, IID)

write.table(ID_KEEP,"/home/jupyter/notebooks/bin/LRRK2/ID_KEEP.txt", row.names = F, col.names = F, quote = F)

# Genetic QC<a href="#Genetic-QC" class="anchor-link"></a>

In [ ]:
## Extracting individuals to keep plink
%%bash
--bfile /home/jupyter/notebooks/bin/genetic_data/AMP --keep ID_KEEP.txt \
--make-bed --out LRRK2_CONTROL 

## QC #Sample 
plink --bfileLRRK2_CONTROL --geno 0.05 --indep-pairwise 50 5 0.05 --maf 0.05 --out \
pruning plink 

--bfile LRRK2_CONTROL --extract pruning.prune.in \ 
--make-bed --out pruned_data plink --bfile pruned_data --het --out prunedHet 

awk '{if (\$6 \<= -0.15) print \$0 }' prunedHet.het > outliers1.txt 
awk '{if (\$6 \>= 0.15) print \$0 }' prunedHet.het > outliers2.txt
cat outliers1.txt outliers2.txt > HETEROZYGOSITY_OUTLIERS.txt 
cut -f 1,2 HETEROZYGOSITY_OUTLIERS.txt > all_outliers.txt 

plink --bfile LRRK2_CONTROL --remove all_outliers.txt \ 
--make-bed --out LRRK2_CONTROL.het

plink --bfile LRRK2_CONTROL.het \
--mind 0.05 --make-bed --out LRRK2_CONTROL.het.mind 
#Variant 
plink --bfile LRRK2_CONTROL.het.mind --geno 0.05 --hwe 1e-6 --maf 0.01 \
--make-bed --out LRRK2_CONTROL.het.mind.geno.maf.hwe 

#IBD 
plink2 --bfile LRRK2_CONTROL.het.geno.maf.hwe --extract pruned.prune.in \
--king-table-filter 0.1 --make-king-table --out IBD

awk '{print \$1, \$2}' IBD.kin0 > IBD_remove.txt

plink --bfile LRRK2_CONTROL.het.geno.maf.hwe --make-bed --out LRRK2_CONTROL.het.geno.maf.hwe.IBD --remove IBD_remove.txt 
#Ancestry

plink2 --bfile /home/jupyter/notebooks/bin/LRRK2/QC/LRRK2_CONTROL.het.geno.maf.hwe.IBD \ 
--make-bed --out cohortPreMerge --rm-dup exclude-all --autosome --snps-only --mac 2 plink2 --alt1-allele \
/home/jupyter/notebooks/bin/genetic_data/ANCESTRY/reference.bim 5 2

--bfile cohortPreMerge --extract /home/jupyter/notebooks/bin/genetic_data/ANCESTRY/reference.bim \
--make-bed --out cohortToMergeHapmap --update-map \
/home/jupyter/notebooks/bin/genetic_data/ANCESTRY/reference.bim 4 2

plink --bfile cohortToMergeHapmap --bmerge \
/home/jupyter/notebooks/bin/genetic_data/ANCESTRY/reference --out
trymerge 

plink --bfile trymerge --bmerge /home/jupyter/notebooks/bin/genetic_data/ANCESTRY/reference --geno 0.05 \
--hwe 1e-6 --maf 0.01 --make-bed --out cohortAndHapmap

plink --bfile cohortAndHapmap --indep-pairwise 50 5 0.05 --out prune 

plink --bfile cohortAndHapmap --extract prune.prune.in  --make-bed  --out \
pruned plink --bfile pruned --out cohortAndHapmap_PCA --pca

In [ ]:
## Make a Scree Plot
dfpcaVal = pd.read_csv('/home/jupyter/notebooks/bin/LRRK2/ANCESTRY/cohortAndHapmap_PCA.eigenval', header=None)
dfpcaVal.plot.line()
plt.title('Scree Plot')
plt.show()

In [ ]:
%%R

# Making a Scree plot

# Read in the PCA Eigenvalues and Eigenvectors
print("Read in pca.eigenval files from PLINK")
eigenval <- read.delim("~/notebooks/bin/LRRK2/ANCESTRY/cohortAndHapmap_PCA.eigenval", sep ="\t", header = F, stringsAsFactors = F)

# Update column names
colnames(eigenval)[1] <- "Eigenvalues"
eigenval$PC <- as.numeric(rownames(eigenval))
eigenval$VarianceExplained <- eigenval$Eigenvalues/sum(eigenval$Eigenvalues)*100

# Keeping only the first 10 PCs
eigenval2 <- head(eigenval,10)

# Generating the plot
scree <- ggplot(data = eigenval2, aes(x = PC, y = VarianceExplained)) +
  geom_line() +
  geom_point() +
  theme_bw() +
  theme(panel.grid.minor = element_blank()) +
  scale_x_continuous(name = "Principal Components", breaks = seq(0,10,1), limits = c(NA,10)) +
  scale_y_continuous(name = "Percent (%) Variance Explained", breaks = seq(0,50,5), limits = c(0,50)) +
  ggtitle("Scree Plot") +
  theme(plot.title = element_text(hjust = 0.5, face = "bold"))

scree

In [ ]:
# PC plot
df = pd.read_csv('~/notebooks/bin/LRRK2/ANCESTRY/cohortAndHapmap_PCA.eigenvec', delim_whitespace=True, header = None)
df.columns=['FID', 'IID', 'PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6', 'PC7', 'PC8', 'PC9', 'PC10', 'PC11', 'PC12', 'PC13', 'PC14', 'PC15', 'PC16', 'PC17', 'PC18', 'PC19', 'PC20']

df.info()

In [ ]:
df[['ID', 'Population', 'Continent']] = df.IID.str.split('_', expand=True)
df['Population']=df.Population.where(pd.notna(df.Continent), '"STUDY"')
df.Continent.where(pd.notna(df.Continent), '"STUDY"', inplace=True)
colors = ['pink','green', 'purple', 'blue']
fig, ax = plt.subplots(1, 1)
for i, (j, group) in enumerate(df.groupby('Continent')):
    if j=='"STUDY"':
        ax.scatter(x=group.PC1, y=group.PC2, color=colors[i], label=j, s = 10, alpha = 1)
    else:
        sns.kdeplot(x=group.PC1, y=group.PC2, n_levels=3, ax=ax, label=j, color=colors[i], alpha=0.8)
plt.title('AMP cohort')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0, fontsize=11)
plt.show()

In [ ]:
# Infer ancestry
dfpop = df.pivot_table(index='Continent', values=['PC1', 'PC2'], aggfunc=['mean', 'std'])

# Get the threshold table of mean +/- 6SD
def funcThres(x):
    lwl = x['mean'] - 6 * x['std']
    hgl = x['mean'] + 6 * x['std']
    return pd.Series({'lwl':lwl, 'hgl':hgl})
thres = dfpop.apply(funcThres, axis=1)

# function to infer ancestry
def funcInfPop(x):
    if x.Continent != '"STUDY"':
        InfPop = 'REF'
    else:
        InfPop = 'ADMIX'
        for continent in ['EUROPE', 'ASIA', 'AFRICA']:
            if (thres.loc[continent, 'lwl']['PC1'] < x.PC1) & \
              (x.PC1 < thres.loc[continent, 'hgl']['PC1']) & \
              (thres.loc[continent, 'lwl']['PC2'] < x.PC2) & \
              (x.PC2 < thres.loc[continent, 'hgl']['PC2']):
                    InfPop = continent
    return InfPop
df['InfPop'] = df.apply(funcInfPop, axis=1)

In [ ]:
# Europeans
dfpca_euro = df[(df.Continent=='EUROPE') | (df.InfPop=='EUROPE')]
colors = ['pink', 'black', 'blue', 'purple', 'orange', 'green', 'red']
fig, ax = plt.subplots(1, 1)
for i, (j, group) in enumerate(dfpca_euro.groupby('Population')):
    if j=='"STUDY"':
        ax.scatter(x=group.PC1, y=group.PC2, color=colors[i], label=j, s = 10, alpha = 1)
    else:
        ax.scatter(x=group.PC1, y=group.PC2, color=colors[i], label=j, s = 10, alpha = 1)
#         sns.kdeplot(x=group.PC1, y=group.PC2, n_levels=3, ax=ax, label=j, color=colors[i], alpha=0.8)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0, fontsize=11)
plt.show()

In [ ]:
# Europeans 
    # PC3 is the determinant of AJ (refer to the AMP-PD genetic resource paper...)
fig, ax = plt.subplots(1, 1)
for i, (j, group) in enumerate(dfpca_euro.groupby('Population')):
    if j=='"STUDY"':
        ax.scatter(x=group.PC1, y=group.PC3, color=colors[i], label=j, s = 10, alpha = 1)
    else:
        ax.scatter(x=group.PC1, y=group.PC3, color=colors[i], label=j, s = 10, alpha = 1)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0, fontsize=11)
plt.show()

In [ ]:
# Create ancestry files
df.loc[df.InfPop!='REF', ['FID', 'IID', 'InfPop'] + [f'PC{i+1}' for i in range(10)]].to_csv('~/notebooks/bin/genetic_data/ANCESTRY/genetic_ancestry_all_pca.csv', index=False)
for continent in ['EUROPE', 'ASIA', 'AFRICA', 'ADMIX']:
    t = df.loc[df.InfPop==continent, ['FID', 'IID']]
    print(t.shape)
    t.to_csv(f'~/notebooks/bin/LRRK2/ANCESTRY/{continent}.txt', index=False, sep='\t')

In [ ]:
# Keeping only Europeans
!plink --bfile /home/jupyter/notebooks/bin/LRRK2/QC/LRRK2_CONTROL.het.geno.maf.hwe.IBD --keep EUROPE.txt --make-bed --out LRRK_CONTROL_QC

In [ ]:
%%R 

# Removing palindromes

bim = read.table("~/notebooks/bin/LRRK2/LRRK_CONTROL_QC.bim", header=F)

#Get indices of A/T and G/C SNPs
w = which((bim$V5=="A" & bim$V6=="T") |
(bim$V5=="T" & bim$V6=="A") |
(bim$V5=="C" & bim$V6=="G") |
(bim$V5=="G" & bim$V6=="C"))

#Extract A/T and G/C SNPs
at.cg.snps = bim[w,]

In [ ]:
%%R

write.table(at.cg.snps$V2,"~/notebooks/bin/LRRK2/at-cg-snps.txt", row.names = F, col.names = F, quote = F)

# Heritability<a href="#Genetic-QC" class="anchor-link"></a>

## Install GCTA

In [ ]:
# Upload the zip file linux version
# Move to data_temp working directory
shell_do(f'gsutil -mu gp2-ipdgc-hackathon cp gs://fc-secure-90f2e0a1-f2e0-4d16-bdf0-ec7fb247a6d3/gcta_1.93.2beta.zip ~/notebooks/bin/data_temp/gcta_temp.zip')

In [ ]:
%%bash
# Unzip the GCTA file
cd '/home/jupyter-user/notebooks/bin/data_temp'
unzip gcta_temp.zip

## Run GCTA with AMP-PD data

In [ ]:
%%bash
# Now try running GRM based on segmented LD scores
# Step 1: segment based LD score
# First split plink files per chromosome to run faster

# Run on all autosomes
cd /home/jupyter-user/notebooks/bin/data_temp/
for chnum in {1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22};
do
    /home/jupyter-user/notebooks/bin/gcta_1.93.2beta/gcta64 \
    --bfile /home/jupyter-user/notebooks/bin/data_temp/target_chr"$chnum" --maf 0.05 \
    --ld-score-region 200 --thread-num 20 \
    --out /home/jupyter-user/notebooks/bin/data_temp/target_segmented_chr"$chnum"
done

In [ ]:
%%R
# Step 2: stratify the SNPs by LD scores of individual SNPs in R
require(dplyr)
require(data.table)
require(tidyverse)
file.names <- dir("/home/jupyter-user/notebooks/bin/data_temp/",pattern=".*score.ld")
file.dfs <- lapply(file.names, fread) %>% lapply(data.frame) %>% rbind()
combined <- bind_rows(file.dfs)
write.table(combined, file="/home/jupyter-user/notebooks/bin/data_temp/all_chr.score.ld", quote=FALSE,row.names=F,sep="\t")

lds_seg <- combined
quartiles=summary(lds_seg$ldscore_SNP)
lb1 = which(lds_seg$ldscore_SNP <= quartiles[2])
lb2 = which(lds_seg$ldscore_SNP > quartiles[2] & lds_seg$ldscore_SNP <= quartiles[3])
lb3 = which(lds_seg$ldscore_SNP > quartiles[3] & lds_seg$ldscore_SNP <= quartiles[5])
lb4 = which(lds_seg$ldscore_SNP > quartiles[5])
lb1_snp = lds_seg$SNP[lb1]
lb2_snp = lds_seg$SNP[lb2]
lb3_snp = lds_seg$SNP[lb3]
lb4_snp = lds_seg$SNP[lb4]
write.table(lb1_snp, "/home/jupyter-user/notebooks/bin/data_temp/snp_group1.txt", row.names=F, quote=F, col.names=F)
write.table(lb2_snp, "/home/jupyter-user/notebooks/bin/data_temp/snp_group2.txt", row.names=F, quote=F, col.names=F)
write.table(lb3_snp, "/home/jupyter-user/notebooks/bin/data_temp/snp_group3.txt", row.names=F, quote=F, col.names=F)
write.table(lb4_snp, "/home/jupyter-user/notebooks/bin/data_temp/snp_group4.txt", row.names=F, quote=F, col.names=F)

In [ ]:
%%bash
cd /home/jupyter-user/notebooks/bin/data_temp/
# Step 3: making GRMs using SNPs stratified into different groups
# Note: GRM is computed using the SNPs on the autosome
for chnum in {1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22};
do
    /home/jupyter-user/notebooks/bin/gcta_1.93.2beta/gcta64 --bfile target_chr"$chnum" \
    --extract snp_group1.txt --make-grm --out chr"$chnum"_group1
    /home/jupyter-user/notebooks/bin/gcta_1.93.2beta/gcta64 --bfile target_chr"$chnum" \
    --extract snp_group2.txt --make-grm --out chr"$chnum"_group2
    /home/jupyter-user/notebooks/bin/gcta_1.93.2beta/gcta64 --bfile target_chr"$chnum" \
    --extract snp_group3.txt --make-grm --out chr"$chnum"_group3
    /home/jupyter-user/notebooks/bin/gcta_1.93.2beta/gcta64 --bfile target_chr"$chnum" \
    --extract snp_group4.txt --make-grm --out chr"$chnum"_group4
done

In [ ]:
%%bash
cd /home/jupyter-user/notebooks/bin/data_temp/
# Step 4: REML analysis with multiple GRMs
# format of multi_grm.txt (no headline; each line represents the prefix of a GRM file)
ls *group*.grm.bin | sed 's/.grm.bin//g' > multi_GRMs.txt
/home/jupyter-user/notebooks/bin/gcta_1.93.2beta/gcta64 --reml --mgrm multi_GRMs.txt --pheno /home/jupyter-user/notebooks/bin/data_temp/covs_Mike.txt --out all_chr_reml --thread-num 20

# Create risk scores using Plink and visualize<a href="#Genetic-QC" class="anchor-link"></a>

In [ ]:
# remove duplicate IDs and reformat to fit with base dataset

%%bash
plink --bfile LRRK_CONTROL_QC.nopalindrome --rm-dup force-first --set-all-var-ids @:# \
--make-bed --out FINAL_FILE 

# Calculating GRS 

plink --bfile FINAL_FILE --score /home/jupyter/notebooks/bin/Nalls2019_GP2_hg38.txt list-variants --out PD_GRS


In [ ]:
%%R

# Reading in the GRS file 

PROFILE <- read.table("/home/jupyter/notebooks/bin/LRRK2/PD_GRS.profile", header = T) 
data <- merge(PROFILE, FINAL_COV, by = c("FID", "IID")) %>%
distinct(FID, .keep_all=TRUE)

In [ ]:
%%R

# Standardizing
meanControls <- mean(data$SCORE[data$DIAGNOSIS1 == 0])
sdControls <- sd(data$SCORE[data$DIAGNOSIS1 == 0])
data$zSCORE <- (data$SCORE - meanControls)/sdControls

head(data)

In [ ]:
%%R

LRRK2_CONTROL <- data %>%
filter(DIAGNOSIS == "LRRK2_PD" | DIAGNOSIS == "CONTROL")

GRS_LRRK2_CONTROL <- glm(DIAGNOSIS1 ~ zSCORE + SEX + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10, family="binomial", data = LRRK2_CONTROL)
summary(GRS_LRRK2_CONTROL)

In [ ]:
%%R

exp(coef(GRS_LRRK2_CONTROL))

In [ ]:
%%R
exp(confint(GRS_LRRK2_CONTROL))

In [ ]:
%%R

LRRK2NOPD_CONTROL <- data %>%
filter(DIAGNOSIS == "LRRK2_NOPD" | DIAGNOSIS == "CONTROL") %>%
mutate(DIAGNOSIS1 = ifelse(DIAGNOSIS1 == 2, 1, DIAGNOSIS1)) #recoding LRRK2_NOPD from 2 to 1 for regression

GRS_LRRK2NOPD_CONTROL <- glm(DIAGNOSIS1 ~ zSCORE + SEX + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10, family="binomial", data = LRRK2NOPD_CONTROL)
summary(GRS_LRRK2NOPD_CONTROL)

In [ ]:
%%R

exp(coef(GRS_LRRK2NOPD_CONTROL))

In [ ]:
%%R
exp(confint(GRS_LRRK2NOPD_CONTROL))

In [ ]:
%%R

LRRK2_CARRIERS <- data %>%
filter(DIAGNOSIS == "LRRK2_NOPD" | DIAGNOSIS == "LRRK2_PD") %>%
mutate(DIAGNOSIS1 = ifelse(DIAGNOSIS1 == 2, 0, DIAGNOSIS1)) #recoding LRRK2_NOPD from 2 to 0 for regression

GRS_LRRK2_CARRIERS <- glm(DIAGNOSIS1 ~ zSCORE + SEX + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10, family="binomial", data = LRRK2_CARRIERS)
summary(GRS_LRRK2_CARRIERS)

In [ ]:
%%R

exp(coef(GRS_LRRK2_CARRIERS))

In [ ]:
%%R
exp(confint(GRS_LRRK2_CARRIERS))

In [ ]:
%%R

p <- ggplot(data, aes(x= reorder(as.factor(DIAGNOSIS), zSCORE), y=zSCORE, fill=as.factor(DIAGNOSIS))) +
  geom_violin(trim=FALSE)

p2 <- p+geom_boxplot(width=0.4, fill="white" ) + theme_minimal()

p3 <- p2 + ylab("PD GRS (Z-transformed)") + xlab("Diagnoses") + theme(legend.position = "none")


p3

In [ ]:
%%R

data %>%
  group_by(DIAGNOSIS) %>%
  summarise_at(vars(SCORE), list(mean = mean, sd = sd))

In [ ]:
%%R 

kruskal.test(SCORE ~ DIAGNOSIS, data = data)

In [ ]:
%%R

pairwise.wilcox.test(data$SCORE, data$DIAGNOSIS,
                 p.adjust.method = "BH")